In [6]:
import requests
from bs4 import BeautifulSoup as bs
import re
import sys
import json
import urllib.request
import pandas as pd
import numpy as np
from model import *

Read words you want to scrape

In [23]:
urls = []
with open("words.txt", "r") as f:
    data = f.read().split("\n")
    
    for word in data:
        urls.append(f"http://5.vndic.net/index.php?word={word}&dict=en_vi")
print(urls)

['http://5.vndic.net/index.php?word=the&dict=en_vi', 'http://5.vndic.net/index.php?word=be&dict=en_vi', 'http://5.vndic.net/index.php?word=and&dict=en_vi', 'http://5.vndic.net/index.php?word=of&dict=en_vi', 'http://5.vndic.net/index.php?word=a&dict=en_vi', 'http://5.vndic.net/index.php?word=in&dict=en_vi', 'http://5.vndic.net/index.php?word=to&dict=en_vi', 'http://5.vndic.net/index.php?word=have&dict=en_vi', 'http://5.vndic.net/index.php?word=to&dict=en_vi', 'http://5.vndic.net/index.php?word=it&dict=en_vi', 'http://5.vndic.net/index.php?word=I&dict=en_vi', 'http://5.vndic.net/index.php?word=that&dict=en_vi', 'http://5.vndic.net/index.php?word=for&dict=en_vi', 'http://5.vndic.net/index.php?word=you&dict=en_vi', 'http://5.vndic.net/index.php?word=he&dict=en_vi', 'http://5.vndic.net/index.php?word=with&dict=en_vi', 'http://5.vndic.net/index.php?word=on&dict=en_vi', 'http://5.vndic.net/index.php?word=do&dict=en_vi', 'http://5.vndic.net/index.php?word=say&dict=en_vi', 'http://5.vndic.net/i

Support function

In [20]:
def save(before, after):
    urllib.request.urlretrieve(before, after)

def trimAudio(audio):
    return re.findall(r"mp3.php\?id=\w+\&dir=\d+&lang=en\&", audio)[0]

In [52]:
# block = None
#     meaning = None
#     # for tr in trs:
#     #     print(tr)
#     for i, tr in enumerate(trs):
            
#         font = tr.find_all("font")
#         content = tr.getText()
        
#         if font == []:
#             if meaning:
#                 block.addMeaning(meaning)
#             meaning = Meaning(content)
#         else :
#             if font[0]["color"] == "#0033CC":
#                 if block:
#                     vocab.addTranslation(block)
#                 block = Block(content)
#             elif font[0]["color"] == "#CC3300":
#                 try:
#                     meaning.addExample(Example(
#                         content,
#                         trs[i + 1].getText()
#                     ))
#                 except:
#                     pass
#             elif font[0]["color"] == "#74726D":
#                 continue
#     if meaning:
#         block.addMeaning(meaning)
#     if block:
#         vocab.addTranslation(block)

Build scraping function

Strip() left and right of string

Remove "Xem" error in string

Remove duplicate in string


In [28]:

vocabularies = []
errorURL = []
def CRAWL(url):
    # Get from url
    try:
        r = requests.get(url)
    except:
        errorURL.append(url)
        return "ERROR!"
    
    
    soup = bs(r.text)
    word = soup.select(".thisword")[0].getText()
    
    # Scrape image
    try:
        image_api = "http://5.vndic.net/" + str(soup.find_all("itemprop")[0].find_all("img")[0]["src"])
        image = f"./Image/{word}.png"
        save(image_api, image)
    except:
        image_api = ""
        image = ""
    
    
    #Scrape audio
    try:
        audio_api = "http://5.vndic.net/" + str(trimAudio(soup.find_all(style ="cursor:pointer;")[0]["onclick"]))
        audio = f"./MP3/{word}.mp3"
        save(audio_api, audio)
    except:
        audio_api = ""
        audio = ""
    
    
    try:
        trs = soup.select(".maincontent")[0].find_all("tr")    
    except:
        trs = []
    
    if trs == []:
       
        return "ERROR!"
    
    spelling = trs[0].getText().strip()

    vocab = Vocabulary(word, image_api, image, audio_api, audio, spelling)

    block = None
    meaning = None
    # for tr in trs:
    #     print(tr)
    try:
        for i, tr in enumerate(trs):
            # print(tr)
            img = tr.find_all("img")
            
            if img:
                src = img[0]["src"]
                # print(src)
                if src == "img/dict/809C2811.png":
                    break
            
            
            content = tr.getText().strip()
            
            if src == "img/dict/CB1FF077.png":
                if meaning:
                    block.addMeaning(meaning)
                meaning = Meaning(content)
            
            elif src == "img/dict/46E762FB.png":
                if block and meaning:
                    block.addMeaning(meaning)
                    meaning = None
                if block and block.meanings:
                    vocab.addTranslation(block)
                block = Block(content)
            elif src == "img/dict/9F47DE07.png":
                if tr.find_all("font")[0]["color"] == "#74726D":
                    continue
                
                meaning.addExample(Example(
                    content,
                    trs[i + 1].getText().strip()
                ))
                
            
                
        if meaning:
            block.addMeaning(meaning)
        if block and block.meanings:
            vocab.addTranslation(block)
    except:
        return "ERROR!"

    vocabularies.append(vocab.toObj())
    # print(json.dumps(vocab.__dict__))

    # for block in vocab.translation:
    #     print(block)
    # print(word)
    
    return "SUCCESSFULL!"



Scraping vocabulary

In [30]:

for i, url in enumerate(urls):
    res = CRAWL(url)
    print(i, res)    
        
with open('vocabulary.json', 'w', encoding='utf-16') as f:
    json.dump(vocabularies, f, ensure_ascii=False, indent=4)
# print(vocabularies)

Testing words after scraping

In [18]:
with open("vocabulary.json", encoding= 'utf-16') as f:
    data = json.load(f)

df = pd.DataFrame(data)
df.to_csv("vocabulary.csv")
df["translation1"][0]

{'type': 'danh từ',
 'meanings': [{'meaning': 'đòn, cú đánh trúng (đấm, bắn, ném)',
   'examples': [{'origin': 'a clever hit', 'meaning': 'một đòn hay'},
    {'origin': 'three hits and five misses',
     'meaning': 'ba đòn trúng năm đòn hụt'}]},
  {'meaning': 'việc thành công; việc đạt kết quả; việc may mắn',
   'examples': [{'origin': 'the new play is quite a hit',
     'meaning': 'bản kịch mới rất thành công'}]},
  {'meaning': 'lời chỉ trích cay độc, nhận xét gay gắt',
   'examples': [{'origin': 'that was a hit at me',
     'meaning': 'câu đó nhằm chỉ trích tôi đấy'}]}]}